In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import numpy.typing as npt

In [ ]:
from sklearn import model_selection
from cardiac_rythm import preprocessing

file_path = "/home/halli/ux/BMDLab/matlab/resprog/GUI/CleanCutsDL/cutDataCinCTTI_rev_v2.mat"

rng = np.random.RandomState(0)
# Load data and create dataframe
df = preprocessing.load_data(file_path)

x = np.stack(df["s_ecg"].to_numpy())
x = x.reshape((*x.shape, 1))
y = df["c_label"].to_numpy()
y = y - 1  # 0-4 instead of 1-5

x_train, x_test, y_train, y_test = model_selection.train_test_split(
    x,
    y,
    test_size=0.1,
    random_state=rng,
    stratify=y,
)

# x_train, y_train = preprocessing.replicate_data(x_train, y_train)

In [ ]:
import tensorflow as tf
from cardiac_rythm.models import CNN, CNNConfig


def get_model(
    filters,
    kernels,
    strides,
    pool,
    fc_end,
    dropout,
) -> CNN:
    model_config = CNNConfig(
        filters,
        kernels,
        strides,
        pool,
        "valid",
        fc_end,
        dropout,
    )
    model = CNN(model_config)
    opt = tf.keras.optimizers.Adam(
        learning_rate=0.001,
        # weight_decay=0.001 / fit_settings.epochs,
    )
    model.compile(
        opt,
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(
    build_fn=get_model,
    verbose=1,
    epochs=100,
    batch_size=32,
)
grid = dict(
    filters=[[5, 10], [10, 5]],
    kernels=[[5, 10], [10, 5]],
    strides=[[1, 1], [2, 2]],
    pool=[[0, 0], [[2, 2], [2, 2]]],
    fc_end=[[32, 64], [64, 128]],
    dropout=[0.3, 0.5],
)


searcher = RandomizedSearchCV(
    estimator=model,
    n_jobs=2,
    cv=3,
    param_distributions=grid,
    scoring="accuracy",
)

In [ ]:
searcher.fit(x_train, y_train, validation_data=())

In [ ]:
searcher.best_params_

In [ ]:
from keras_tuner import RandomSearch, HyperParameters


def build_model(hp: HyperParameters) -> CNN:
    filters = []
    kernels = []
    strides = []
    pool = []
    for i in range(2):
        filters.append(hp.Choice(f"filters{i}", [5, 10, 15, 20]))
        kernels.append(hp.Choice(f"kernels{i}", [5, 10, 15, 20]))
        strides.append(2)
        pool.append([2, 2])

    model_config = CNNConfig(
        filters,
        kernels,
        strides,
        pool,
        "valid",
        fc_end=[64, 128],
        dropout=hp.Float("dropout", 0.1, 0.9, step=0.1),
    )
    print(model_config)
    model = CNN(model_config)
    opt = tf.keras.optimizers.Adam(
        learning_rate=0.001,
        # weight_decay=0.001 / fit_settings.epochs,
    )
    model.compile(
        opt,
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [ ]:
x_train.shape

In [ ]:
import pandas as pd


yt = pd.get_dummies(y_train)
yv = pd.get_dummies(y_test)

tuner = RandomSearch(build_model, objective="val_loss", max_trials=5)
tuner.search(
    x_train,
    yt,
    validation_data=(x_test, yv),
    batch_size=32,
    epochs=100,
)

In [ ]:
x = np.stack(df["s_ecg"].to_numpy())
x = x.reshape((*x.shape, 1))
y = df["c_label"].to_numpy()
y = y - 1  # 0-4 instead of 1-5

x_train, x_test, y_train, y_test = model_selection.train_test_split(
    x,
    y,
    test_size=0.1,
    random_state=rng,
    stratify=y,
)

In [ ]:
import copy
import pickle
from keras_tuner import HyperModel
from keras_tuner.src.engine import tuner_utils
from sklearn.model_selection import StratifiedKFold

from cardiac_rythm.visualize import visualize_history, visualize_test_result


class RandomSearchOptimization(RandomSearch):
    def __init__(
        self,
        n_folds: int = 10,
        rng: np.random.RandomState = np.random.RandomState(0),
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.skf = StratifiedKFold(n_splits=n_folds, random_state=rng, shuffle=True)

    def run_trial(self, trial, *args, **kwargs):
        """Override `run_trial` to implement cross validation."""
        X_train, y_train, X_test, y_test, *remaining_args = args

        trial_dir = self.get_trial_dir(trial_id=trial.trial_id)
        # Not using `ModelCheckpoint` to support MultiObjective.
        # It can only track one of the metrics to save the best model.
        saved_model = self._get_checkpoint_fname(trial.trial_id)
        model_checkpoint = tuner_utils.SaveBestEpoch(
            objective=self.oracle.objective,
            filepath=saved_model,
        )
        original_callbacks = kwargs.pop("callbacks", [])

        # Run the training process multiple times.
        histories = []
        for execution in range(self.executions_per_trial):
            total_acc = 0.0
            total_val_acc = 0.0
            total_loss = 0.0
            total_val_loss = 0.0
            for fold, (train_index, val_index) in enumerate(self.skf.split(X_train, y_train)):
                xt, xv = X_train[train_index], X_train[val_index]
                yt, yv = y_train[train_index], y_train[val_index]
                yt = pd.get_dummies(yt)
                yv = pd.get_dummies(yv)
                copied_kwargs = copy.copy(kwargs)
                callbacks = self._deepcopy_callbacks(original_callbacks)

                # Add fold to tensorboard callback
                for callback in callbacks:
                    if isinstance(callback, tf.keras.callbacks.TensorBoard):
                        callback.log_dir += f"/fold_{fold}"

                self._configure_tensorboard_dir(callbacks, trial, execution)
                callbacks.append(tuner_utils.TunerCallback(self, trial))
                # Only checkpoint the best epoch across all executions.
                callbacks.append(model_checkpoint)
                copied_kwargs["callbacks"] = callbacks

                hp = trial.hyperparameters
                model: CNN = self._try_build(hp)
                history = self.hypermodel.fit(
                    hp,
                    model,
                    xt,
                    yt,
                    validation_data=(xv, yv),
                    *remaining_args,
                    **copied_kwargs,
                )

                # Test the model and save conf mat, accuracy and loss
                # TODO: Or just do it on the BEST model?
                visualize_history(history.history, f"{trial_dir}/{fold}.svg")
                # Dump history in case we want to plot it.
                with open(f"{trial_dir}/history_dict_{fold}", "wb") as f:
                    pickle.dump(history.history, f)
                # Dump the model config in a better format
                if fold == 0:
                    with open(f"{trial_dir}/model_config.json", "x") as f:
                        f.write(model.config.to_json(indent=2))

                total_loss += min(history.history["loss"])
                total_val_loss += min(history.history["val_loss"])
                total_acc += max(history.history["accuracy"])
                total_val_acc += max(history.history["val_accuracy"])

            # Store average loss/accuracy
            avg_loss = total_loss / self.skf.get_n_splits()
            avg_val_loss = total_val_loss / self.skf.get_n_splits()
            avg_accuracy = total_acc / self.skf.get_n_splits()
            avg_val_accuracy = total_val_acc / self.skf.get_n_splits()

            histories.append(
                {
                    "loss": avg_loss,
                    "val_loss": avg_val_loss,
                    "accuracy": avg_accuracy,
                    "val_accuracy": avg_val_accuracy,
                }
            )
            model.load_weights(saved_model)
            prediction = np.argmax(model.predict(X_test), axis=1)
            visualize_test_result(y_test, prediction, f"{trial_dir}/conf_mat.svg")

        return histories


class HyperTuner(HyperModel):
    def __init__(self, n_filters: int, **kwargs):
        super().__init__(**kwargs)
        self.n_filters = n_filters

    def build(self, hp: HyperParameters) -> CNN:
        filters = []
        kernels = []
        strides = []
        pool = []
        for i in range(self.n_filters):
            filters.append(hp.Choice(f"filters{i}", [5, 10, 15, 20]))
            kernels.append(hp.Choice(f"kernels{i}", [5, 10, 15, 20]))
            strides.append(2)
            pool.append([2, 2])

        model_config = CNNConfig(
            filters,
            kernels,
            strides,
            pool,
            "valid",
            fc_end=[64, 128],
            dropout=hp.Float("dropout", 0.1, 0.9, step=0.1),
        )

        model = CNN(model_config)
        opt = tf.keras.optimizers.Adam(
            learning_rate=0.001,
            # weight_decay=0.001 / fit_settings.epochs,
        )
        model.compile(
            opt,
            loss="categorical_crossentropy",
            metrics=["accuracy"],
        )
        return model


callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        mode="min",
        verbose=1,
        patience=50,
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir="/tmp/tb",
    ),
]

tuner = RandomSearchOptimization(
    n_folds=2,  # TODO: 10 10 10
    hypermodel=HyperTuner(n_filters=2),
    objective="val_loss",
    max_trials=2,
    # directory=f"{i}",
    project_name="custom_training",
    overwrite=True,  # TODO: FALSE FALSE FALSE! (don't want to lose data after many trials..)
)
tuner.search(
    x_train,
    y_train,
    x_test, 
    y_test,
    batch_size=32,
    epochs=50,
    callbacks=callbacks,
)

In [ ]:
import keras
from cardiac_rythm.hyper_tune import RandomSearchOptimization, HyperTuner
n_filters = 2
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        mode="min",
        verbose=1,
        patience=50,
    ),
    keras.callbacks.TensorBoard(
        log_dir=f"results/{n_filters}/tb",
    ),
]

tuner = RandomSearchOptimization(
    rng,
    n_folds=2,  # TODO: 10 10 10
    hypermodel=HyperTuner(n_filters=n_filters),
    objective="val_loss",
    max_trials=2,
    directory="results",
    project_name=f"{n_filters=}",
    overwrite=True,  # TODO: FALSE FALSE FALSE! (don't want to lose data after many trials..)
)
tuner.search(
    x_train,
    y_train,
    x_test,
    y_test,
    batch_size=32,
    epochs=50,
    callbacks=callbacks,
)
